In [1]:
import numpy as np
from docopt import docopt
from scipy import sparse
import os
import subprocess
from shutil import copyfile
from operator import itemgetter

In [2]:
'''
workflow of the verify_implementation script

verify glove bias
1. I build W and H matrix, build bias term. And then, multply W and H, add bias term, use this matrix as my question
2. store the question matrix nnz term and meta information to ./matrix_folder
3. (this step is not important, because weight is closed when testing bias term)
   build a weight matrix and store its question and nnz term to ./count_folder
4. modified code, let it output the bias term when factorize the matrix
5. read in the answer matrix, test them, to see whether it can output the bias term

verify glove weight
1. I build W and H matrix, multply W and H, use this matrix as my question matrix
2. build a weight matrix, assign one position very high weight, store nnz and meta file in count_folder
3. open glove weight, close glove bias, factorize the matrix
4. read in the answer matrix, test them, to see the error on the position with high weight

'''

'\nworkflow of the verify_implementation script\n\nverify glove bias\n1. I build W and H matrix, build bias term. And then, multply W and H, add bias term, use this matrix as my question\n2. store the question matrix nnz term and meta information to ./matrix_folder\n3. build a weight matrix and store its question and nnz term to ./count_folder\n4. modified code, let it output bias term when factorize the matrix\n5. read in the answer matrix, test them, to see whether it can output the bias term\n\nverify glove weight\n1. I build W and H matrix, multply W and H, use this matrix as my question\n2. build a weight matrix, assign one position very high weight, store nnz and meta file in count_folder\n3. open glove weight, close glove bias, factorize the matrix\n4. 5. read in the answer matrix, test them, to see the error on the position with high weight\n'

In [3]:
'''
input: matrix_size = the size of question matrix which I need to factorize (how many words in total)
input: embedding_rank
input: include_bias (default = 0), set it as 1, when you need to add bias term when generating question matrix.
       only when include_bias = 1, we need to care the W_bias and H_bias in the return object.
       when include_bias = 0, the W_bias and H_bias in the return object are useless.
       
output: W, H, W_bias, H_bias, question_matrix; all of them are np array
        question_matrix[i][j] = W[i] * H[j] + W_bias[i] + H_bias[j]
        W: matrix_size * embedding_rank, np array
        H: embedding_rank * matrix_size, np array
        W_bias: matrix_size, np array
        H_bias: matrix_size, np array
        question_matrix: matrix_size * matrix_size, np array
'''
def build_answer_and_question(matrix_size, embedding_rank, include_bias = 0):
    W = np.random.random((matrix_size, embedding_rank))
    H = np.random.random((embedding_rank, matrix_size))

#     W = W*(W>0.55)
#     H = H*(H<0.45)
    
    W_bias = np.random.random(matrix_size)
    H_bias = np.random.random(matrix_size)
    
    question_matrix = np.dot(W, H)
    
    if include_bias:

        print "inlcude_bias is on"
        question_matrix = np.transpose(question_matrix)
        for i in range(question_matrix.shape[0]):
            question_matrix[i] = np.add(question_matrix[i], W_bias)
        question_matrix = np.transpose(question_matrix)
        
        for i in range(question_matrix.shape[1]):
            question_matrix[i] = np.add(question_matrix[i], H_bias)
    
    
    
    print question_matrix[1][5]
    print (np.dot(W[1], H[:,5]) + W_bias[1] + H_bias[5])
    return W, H ,W_bias, H_bias, question_matrix

In [4]:
'''
input: matrix_size, I need to build a matrix_size * matrix_size weight matrix, everywhere except [1][1] is 1
input: weight_for_exception_term, how large weight you want to assign to the exception position

output: a matrix_size * matrix_size weight matrix, np array, the number in the [1][1] is weight_for_exception_term
'''
def build_weight_matrix(matrix_size, weight_for_exception_term):
    weight_matrix = np.ones((matrix_size, matrix_size))
    weight_matrix[1][1] = 1
    
    return weight_matrix

In [5]:
'''
input is a np array, path_and_name

this function will save the nnz in path_and_name in the following foramt:
x_coordinate y_coordinate value
please note, in the final document, x_coordinate and y_coordinate start from 1, not 0; program will add 1 to each term's x and y coordinate.

'''
def save_nonzero_term_fast(nparray, path_and_name):
    csrmatrix = sparse.csr_matrix(nparray)  
    with open(path_and_name,'w') as f:
        for i in range(len(csrmatrix.indptr)-1):
#             if i % 1000000 == 0:
#                 print i, len(csrmatrix.indptr)-1
            columnIndices=[]
            dataInLine=[]
            columnIndices=csrmatrix.indices[csrmatrix.indptr[i]:csrmatrix.indptr[i+1]] 
            dataInLine=csrmatrix.data[csrmatrix.indptr[i]:csrmatrix.indptr[i+1]]
            for j in range(len(columnIndices)):
                f.write("%d %d %.6f\n"% (i+1, columnIndices[j]+1, dataInLine[j]))         

In [6]:
'''
input is a np array, path_and_name

this function will save the information in path_and_name in the following foramt:
matrix_size matrix_size
nnz_number training.ratings
nnz_number test.ratings
for example:
71290 71290
1289567 training.ratings
1289567 test.ratings
'''
def create_meta_file(nparray, path_and_name):
    csrmatrix = sparse.csr_matrix(nparray)
    nnz_number = csrmatrix.getnnz()
    size = nparray.shape[0]

    with open(path_and_name,'w') as f:
        f.write("%d %d\n" %(size, size))
        f.write("%d %s\n" %(nnz_number, 'training.ratings'))
        f.write("%d %s" %(nnz_number, 'test.ratings'))

In [7]:
'''
this function will read in the left and right matrix in the path_and_name
input: path_and_name
this funciton will read in the path_and_name +'.W' and path_and_name +'.H' file

output: left matrix a and right matrix b
        left matrix a: matrix_size * embedding_rank, np array
        right matrix b: embedding_rank * matrix_size, np array
'''
def read_answer(path_and_name):
    a = np.loadtxt(path_and_name +'.W')
#     print ("a.shape",a.shape)
    b = np.transpose(np.loadtxt(path_and_name +'.H'))
#     print ("b.shape",b.shape)
    return a, b

In [8]:
'''
verify glove weight
set include_bias as 0 when build qutstion matrix
save nnz terms and meta information
build weight matrix and save cooresponding information, assign a very high weight to [1][1] position
factorize the matrix, set G(glove_bias) as 0, set W(glvove_weight) as 1. set x_max as a very high number
read in and delete answer matrix
see the difference in every position between original matrix and rebuilded matrix 
'''

matrix_size = 8
rank = 6

W, H ,W_bias, H_bias, question_matrix = build_answer_and_question(matrix_size, rank, 0)
create_meta_file(question_matrix, "./matrix_folder/meta")
save_nonzero_term_fast(question_matrix, "./matrix_folder/training.ratings")
save_nonzero_term_fast(question_matrix, "./matrix_folder/test.ratings")

weight_matrix = build_weight_matrix(matrix_size, 1000)
create_meta_file(weight_matrix, "./count_folder/meta")
save_nonzero_term_fast(weight_matrix, "./count_folder/training.ratings")
save_nonzero_term_fast(weight_matrix, "./count_folder/test.ratings")

subprocess.check_output(["make"])
subprocess.check_output(["./converter", "./matrix_folder"])
subprocess.check_output(["./omp-pmf-train", "-s", "10", "-n", "10", "-f", "1", "-t", "20", "-q", "1", "-p", "0", "-r", "0.015625", "-l", "0.015625", "-b", "0", "-k", str(rank), "-E", "0", "-X", "1000000", "-W", "1", "-G", "0", "matrix_folder", "count_folder", "test_code"])

W_answer, H_answer = read_answer('test_code-l0.015625-r0.015625-iter20-gweight1-xmax1000000-gbias0.final')
subprocess.check_output(["rm", 'test_code-l0.015625-r0.015625-iter20-gweight1-xmax1000000-gbias0.final.W'])
subprocess.check_output(["rm", 'test_code-l0.015625-r0.015625-iter20-gweight1-xmax1000000-gbias0.final.H'])

print np.subtract(question_matrix, np.dot(W_answer, H_answer))

1.14859811964
2.26181536265
[[ 2.63947283  2.0406623   1.19458877  2.15369068  1.19950848  1.30697975
   1.89603186  1.31834889]
 [ 2.24686787  1.61518499  0.84810509  1.59010401  0.90798864  1.14859812
   1.56842441  1.05412623]
 [ 2.26894776  1.36705681  0.69838035  1.91368497  0.63435601  1.10819921
   1.37179661  0.77100578]
 [ 3.00803645  2.26961125  1.37414111  2.59521223  1.10857258  1.56039249
   1.70545589  1.37932572]
 [ 2.48556907  1.89074542  1.1144728   1.70118599  0.98328166  1.29495298
   1.54848795  1.18431796]
 [ 1.92636216  1.30757743  0.83123069  1.56457634  0.83455125  0.96362978
   1.44189385  1.01705652]
 [ 2.8755597   1.87198297  1.30679197  2.07522847  1.02026398  1.42668388
   1.8393194   1.30124779]
 [ 1.21705964  0.86208757  0.55048013  1.17303518  0.54837343  0.6493422
   0.86623937  0.73323406]]


In [885]:
'''
verify glove bias
set include_bias as 1 when build qutstion matrix
save nnz terms and meta information
build weight matrix and save cooresponding information(this step is useless)
factorize the matrix, set G(glove_bias) as 1, set W(glvove_weight) as 0.
read in and delete answer matrix
see the difference in every position between original matrix and rebuilded matrix 
see the difference between original bias and the bias in my answer
'''
# def main():
#     args = docopt("""
#     Usage: 
#         verify_inner_product.py <representation_shared_path>
#     """)

matrix_size = 8
rank = 5
W, H ,W_bias, H_bias, question_matrix = build_answer_and_question(matrix_size, rank, 1)

# print question_matrix

create_meta_file(question_matrix, "./matrix_folder/meta")
save_nonzero_term_fast(question_matrix, "./matrix_folder/training.ratings")
save_nonzero_term_fast(question_matrix, "./matrix_folder/test.ratings")

create_meta_file(question_matrix, "./count_folder/meta")
save_nonzero_term_fast(question_matrix, "./count_folder/training.ratings")
save_nonzero_term_fast(question_matrix, "./matrix_folder/test.ratings")

subprocess.check_output(["make"])
subprocess.check_output(["./converter", "./matrix_folder"])
subprocess.check_output(["./omp-pmf-train", "-s", "10", "-n", "10", "-f", "1", "-t", "10", "-q", "1", "-p", "0", "-r", "0.015625", "-l", "0.03125", "-b", "0", "-k", str(rank), "-E", "0", "-X", "1", "-W", "0", "-G", "2", "matrix_folder", "count_folder", "test_code"])

W_answer, H_answer = read_answer('test_code-l0.031250-r0.015625-iter10-gweight0-xmax1-gbias2.final')
subprocess.check_output(["rm", 'test_code-l0.031250-r0.015625-iter10-gweight0-xmax1-gbias2.final.W'])
subprocess.check_output(["rm", 'test_code-l0.031250-r0.015625-iter10-gweight0-xmax1-gbias2.final.H'])

print W_bias
print W_answer[:,rank+1]

print H_bias
print H_answer[rank,:]

W_bias_dict = dict([(i, a) for i, a in enumerate(W_bias)])
sort_W_bias_dict = sorted(W_bias_dict.items(), key=itemgetter(1))
print sort_W_bias_dict

W_answer_dict = dict([(i, a) for i, a in enumerate(W_answer[:,rank+1])])
sort_W_answer_dict = sorted(W_answer_dict.items(), key=itemgetter(1))
print sort_W_answer_dict

H_bias_dict = dict([(i, a) for i, a in enumerate(H_bias)])
sort_H_bias_dict = sorted(H_bias_dict.items(), key=itemgetter(1))
# print H_bias_dict

H_answer_dict = dict([(i, a) for i, a in enumerate(H_answer[rank,:])])
sort_H_answer_dict = sorted(H_answer_dict.items(), key=itemgetter(1))
# print H_answer_dict

# print np.subtract(W_bias, W_answer[:,rank+1])
# print np.subtract(H_bias, H_answer[rank,:])

inlcude_bias is on
2.11814568245
2.11814568245
[ 0.742  0.035  0.565  0.788  0.807  0.797  0.974  0.709]
[ 0.025 -0.085  0.064  0.018  0.001  0.001  0.083  0.05 ]
[ 0.884  0.645  0.792  0.581  0.905  0.975  0.075  0.2  ]
[ 0.17   0.148  0.203  0.146  0.222  0.197  0.06   0.105]
[(1, 0.034687117733612549), (2, 0.5646343551039168), (7, 0.70892671481778602), (0, 0.74154172135293617), (3, 0.78792273763768861), (5, 0.79665764254060456), (4, 0.80666487565685507), (6, 0.9739816139185018)]
[(1, -0.084616999999999998), (5, 0.001132), (4, 0.0014790000000000001), (3, 0.017877000000000001), (0, 0.025127), (7, 0.049993000000000003), (2, 0.064140000000000003), (6, 0.082632999999999998)]
